In [1]:
from sklearn.preprocessing import Normalizer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,PoissonRegressor
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import  xgboost  as xg
import pandas as pd 
import numpy as np
import pickle
import os

#### INSERTING DATA

In [2]:
#Loading Datamart
data_dir="C:\\Users\\Arda\\Downloads\\football_master\\Documantation\\Datas\\"
data=pd.read_csv("{}datamart.csv".format(data_dir))

In [3]:
#Selecting played matches
data= data[data["fixture.status.short"]=="FT"].reset_index(drop=True)

In [4]:
#Selecting feature columns
feature_data=data.iloc[:,27:]
feature_data

,Foul_Home,Corner_Kicks_Home,Offsides_Home,Ball_Possession_Rate_Home,Yellow_Cards_Home,Red_Cards_Home,Total_Passes_Home,Accurate_Pass_Rate_Home,Total_Shot_Home,Shot_On_Goal_Rate_Home,...,Rank_Name_last_2_3_Away,Rank_Name_title_Away,Rise_Need_1_point_needed_Away,Rise_Need_3_point_for_catch_Away,Rise_Need_3_point_for_chase_Away,Rise_Need_top_Away,Fall_Prevent_1_point_for_run_Away,Fall_Prevent_3_point_for_run_Away,Fall_Prevent_bottom_Away,Fall_Prevent_comfort_Away
0,0.0,10.0,0.0,0.580,2.0,0.0,460.0,0.840,18.0,0.400,...,0,0,1,0,0,0,0,1,0,0
1,0.0,4.0,1.0,0.540,1.0,0.0,457.0,0.820,11.0,0.250,...,0,0,1,0,0,0,0,1,0,0
2,0.0,9.0,0.0,0.370,2.0,0.0,306.0,0.700,16.0,0.430,...,0,0,1,0,0,0,0,1,0,0
3,0.0,2.0,0.0,0.440,3.0,0.0,368.0,0.780,10.0,0.500,...,0,0,1,0,0,0,1,0,0,0
4,0.0,9.0,1.0,0.540,0.0,0.0,463.0,0.810,12.0,0.360,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68657,18.0,4.5,1.5,0.370,3.0,0.0,279.5,0.700,10.5,0.415,...,0,0,1,0,0,0,1,0,0,0
68658,17.0,3.5,3.0,0.425,2.5,0.0,348.0,0.780,9.0,0.470,...,0,0,1,0,0,0,0,1,0,0
68659,13.5,7.0,1.0,0.585,1.5,0.0,420.0,0.785,13.5,0.340,...,0,0,0,0,1,0,0,1,0,0
68660,9.5,5.0,1.0,0.485,1.0,0.5,366.5,0.785,10.5,0.195,...,0,0,1,0,0,0,1,0,0,0


In [6]:
data[["fixture.id","goals.home","goals.away"]]

,fixture.id,goals.home,goals.away
0,185555,0.0,0.0
1,185556,1.0,1.0
2,185557,0.0,0.0
3,185558,0.0,1.0
4,185559,3.0,3.0
...,...,...,...
68657,1146779,2.0,0.0
68658,1146780,2.0,0.0
68659,1146781,1.0,1.0
68660,1146782,1.0,1.0


In [7]:
#Selecting only columns needed for statistical learning
feature_data_v1=pd.concat([data[["fixture.id","goals.home","goals.away"]],
                            feature_data],axis=1)


In [8]:
#From away and home scores, status of given fixtures result being over 2.5 goals is created.
feature_data_v1["Full_Score"]=feature_data_v1["goals.home"]+feature_data_v1["goals.away"]
feature_data_v1["Over_2.5"]=feature_data_v1["Full_Score"].apply(lambda x: 1 if x>2 else 0)
feature_data_v1

,fixture.id,goals.home,goals.away,Foul_Home,Corner_Kicks_Home,Offsides_Home,Ball_Possession_Rate_Home,Yellow_Cards_Home,Red_Cards_Home,Total_Passes_Home,...,Rise_Need_1_point_needed_Away,Rise_Need_3_point_for_catch_Away,Rise_Need_3_point_for_chase_Away,Rise_Need_top_Away,Fall_Prevent_1_point_for_run_Away,Fall_Prevent_3_point_for_run_Away,Fall_Prevent_bottom_Away,Fall_Prevent_comfort_Away,Full_Score,Over_2.5
0,185555,0.0,0.0,0.0,10.0,0.0,0.580,2.0,0.0,460.0,...,1,0,0,0,0,1,0,0,0.0,0
1,185556,1.0,1.0,0.0,4.0,1.0,0.540,1.0,0.0,457.0,...,1,0,0,0,0,1,0,0,2.0,0
2,185557,0.0,0.0,0.0,9.0,0.0,0.370,2.0,0.0,306.0,...,1,0,0,0,0,1,0,0,0.0,0
3,185558,0.0,1.0,0.0,2.0,0.0,0.440,3.0,0.0,368.0,...,1,0,0,0,1,0,0,0,1.0,0
4,185559,3.0,3.0,0.0,9.0,1.0,0.540,0.0,0.0,463.0,...,1,0,0,0,0,1,0,0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68657,1146779,2.0,0.0,18.0,4.5,1.5,0.370,3.0,0.0,279.5,...,1,0,0,0,1,0,0,0,2.0,0
68658,1146780,2.0,0.0,17.0,3.5,3.0,0.425,2.5,0.0,348.0,...,1,0,0,0,0,1,0,0,2.0,0
68659,1146781,1.0,1.0,13.5,7.0,1.0,0.585,1.5,0.0,420.0,...,0,0,1,0,0,1,0,0,2.0,0
68660,1146782,1.0,1.0,9.5,5.0,1.0,0.485,1.0,0.5,366.5,...,1,0,0,0,1,0,0,0,2.0,0


In [9]:
feature_data_v1[["Full_Score"]].pivot_table(aggfunc="size",index="Full_Score")

Full_Score
0.0      5191
1.0     12421
2.0     16721
3.0     15226
4.0      9827
5.0      5381
6.0      2481
7.0       957
8.0       330
9.0        95
10.0       25
11.0        4
12.0        1
13.0        2
dtype: int64

In [10]:
#Distrubition of goals are skewed to rigth
#In order to ballance distrubition with out damaging cause of the project
# We will assing 6 to over 6 goals

In [11]:
feature_data_v1["Full_Score_adj"]=feature_data_v1["Full_Score"].apply(lambda x: x if x<7 else 6)
feature_data_v1["Full_Score_adj"]

0        0.0
1        2.0
2        0.0
3        1.0
4        6.0
        ... 
68657    2.0
68658    2.0
68659    2.0
68660    2.0
68661    2.0
Name: Full_Score_adj, Length: 68662, dtype: float64

#### Train Test 

In [12]:
#Train and test data selected, accoriding to full score adj distrubition
train,test= train_test_split(feature_data_v1,test_size=0.10,stratify=feature_data_v1["Full_Score_adj"],random_state=42)
train.head()

,fixture.id,goals.home,goals.away,Foul_Home,Corner_Kicks_Home,Offsides_Home,Ball_Possession_Rate_Home,Yellow_Cards_Home,Red_Cards_Home,Total_Passes_Home,...,Rise_Need_3_point_for_catch_Away,Rise_Need_3_point_for_chase_Away,Rise_Need_top_Away,Fall_Prevent_1_point_for_run_Away,Fall_Prevent_3_point_for_run_Away,Fall_Prevent_bottom_Away,Fall_Prevent_comfort_Away,Full_Score,Over_2.5,Full_Score_adj
34414,164586,2.0,1.0,13.5,3.0,1.5,0.495,3.0,0.0,380.0,...,0,0,0,0,1,0,0,3.0,1,3.0
62021,1039447,1.0,0.0,9.5,4.5,2.5,0.380,1.0,0.0,277.5,...,1,0,0,0,0,0,1,1.0,0,1.0
19398,203647,0.0,0.0,0.0,3.0,2.0,0.440,5.5,1.0,334.5,...,0,1,0,0,1,0,0,0.0,0,0.0
10606,209160,3.0,3.0,11.0,4.0,2.0,0.455,2.0,0.0,443.0,...,1,0,0,0,1,0,0,6.0,1,6.0
6144,838291,0.0,0.0,17.0,8.5,1.0,0.585,3.0,0.0,455.5,...,0,1,0,0,1,0,0,0.0,0,0.0


In [22]:
class betting_model:
    
    #Possible ML models initialized, 
    #Binary key for model which learn from binary target("Over_2.5")
    #poisson key for model which learn from count data("Full_Score_adj")
    models={"binary":{"Log_Reg":LogisticRegression(max_iter=10000,random_state=42),
                       "Decistion_Tree":DecisionTreeClassifier(max_depth=5, random_state=42),
                       "Random_Forrest":RandomForestClassifier(random_state=42)},
            "poisson":{"XG_Poisson":xg.XGBModel(objective="count:poisson",random_state=42)}
           }
    
    def __init__(self,train_data,test_data,binary_target,features_dict={},count_target=None):
        
        #train_data:    Selected train data
        #test_data:     Selected test data
        #binary_target: Binary target variable 
        #count_target:  Count target variable
        #features_dict: Variations of features varaibles stored in dictonary
        
        self.train_data=train_data
        self.test_data=test_data
        self.binary_target=binary_target
        self.count_target=count_target
        
        #In addition to selected features, all features also added for learning trials.
        all_cols=[i for i in train_data.columns if i != self.binary_target and i != self.count_target ]
        
        self.features_dict={"all_columns":all_cols}
        
        self.features_dict={**self.features_dict,**features_dict}
        
    
    def fit(self,model,features,mode="binary"):
        #Fit method, fits selected model with choosen training mode and feature columns.
        
        fitted_model=model
        
        if mode=="binary":
            #Fitting the model with train data
            fitted_model.fit(self.train_data[features],self.train_data[self.binary_target])
            #Predicting train data
            predict_train=fitted_model.predict_proba(self.train_data[features])[:,1]
            #Predicting test data
            predict_test=fitted_model.predict_proba(self.test_data[features])[:,1]
            return fitted_model,predict_train,predict_test
            
            
        elif mode=="poisson":
            #Fitting the model with train data
            fitted_model.fit(self.train_data[features],self.train_data[self.count_target])
            #Predicting train data
            predict_train=fitted_model.predict(self.train_data[features])
            #Predicting test data
            predict_test=fitted_model.predict(self.test_data[features])
            return fitted_model,predict_train,predict_test
    
    def metric(self,pred_proba,target,mode='binary'):
        #pred_proba: Predicted propensity for binary target variable and count value for count target variable
        #target:     Target variable
        #mode:       Mode for given target variable 'binary'/'poisson'
        
        metrics={}
        
        if mode == "binary":
            predicts=pd.DataFrame({"Positif_Prob":pred_proba,"Target":target})

        
        elif mode=="poisson":
            #In order to create propensity score for count data,
            #First over 5 goals fixed to 5. 
            #Second predictions dived by 5.With that divide outcome 2.5 would be 0.5(2.5/5)
            
            series=pd.Series(pred_proba).apply(lambda x: 5 if x>5 else x)
            predicts=pd.DataFrame({"Positif_Prob":series.values/5,"Target":target})



        else:

            print("Please choose between binary and poisson")
        #tresholds list for determining recall at given levels   
        tresholds=[0.5,0.7,0.8,0.9]
        for treshold in tresholds:
            
            #Predicts over given treshold
            pozitif_treshold=predicts[predicts.Positif_Prob>treshold]
            #Positive recall for given treshold
            pozitif_acc=pozitif_treshold.Target.sum()/pozitif_treshold.shape[0]
            #Predicts below given treshold-1 (reason: positive treshold: 0.7, negative treshold: 1-0.7=0.3)
            negatif_treshold=predicts[predicts.Positif_Prob<1-treshold]
            #Negative recall for given treshold. 
            #Becouse negative target is 0, negative recall= 1-positive recall
            negatif_acc=1-negatif_treshold.Target.sum()/negatif_treshold.shape[0]
            #Recall results of both kind of treshold recorded to metrics dictonary.
            metrics["pozitif_acc_treshold_{}".format(treshold)]=[pozitif_acc]
            metrics["negatif_acc_treshold_{}".format(treshold)]=[negatif_acc]
        
        return metrics

    def trails(self,ensemble=None,record=True):
        
        #ensemble: Initiliazed as true, if combinations of model wants to be used.
        #Records:  Initiliazed as true, if models and their results wants to be saved.
        trial_log=pd.DataFrame()
        model_preds={}
        
        if record:
            #File direction of records
            record_dir="ml_records"
            try:
                os.mkdir(record_dir)
            except: 
                pass
        
        #Iterated over target modes(binary/poisson)
        for mod in self.models.keys():
            
            mod_dict=self.models[mod]
            
            #Iterated over models with in selected mode.
            for m in mod_dict.keys():
                
                print(m)
                
                #Iterated over given feature columns
                for feat in self.features_dict:
                    print(feat)
                    
                    #Dictionary for each ml reacord
                    records={}
                    #Selected feature columns
                    features=self.features_dict[feat]
                    
                    #Output of fit method.
                    #fitted_model: trained model
                    #pred_train: prediction propensity of train data
                    #pred_test: prediction propensity of test data
                    fitted_model,pred_train,pred_test= self.fit(mod_dict[m],features,mode=mod)
                    
                    #Selected model name
                    records["Model_Name"]=[m]
                    
                    #Given models structure ("single"/"ensemble")
                    records["type"]=["Single"]
                    
                    #Key name of feature columns set
                    records["feat_select"]=[feat]
                    
                    #Target variable distrubition type ("binary"/"poisson")
                    records["Target_Dist"]=[mod]
                    
                    #Record dictionary for train data
                    train_records={"Data_Type":"Train"}
                    
                    #Records add to train records
                    train_records= {**records,**train_records}
                    
                    #Positive and negative recall scores of given model at train data calculated.
                    train_records={**train_records,**self.metric(pred_train,self.train_data[self.binary_target],mode=mod)}
                    
                    #Train records of given model added to trial logs
                    trial_log= pd.concat([trial_log,pd.DataFrame.from_dict(train_records)])
                    
                    #Record dictionary for test data
                    test_records={"Data_Type":"Test"}
                    
                    #Records add to test records
                    test_records= {**records,**test_records}
                    
                    #Positive and negative recall scores of given model at test data calculated.
                    test_records={**test_records,**self.metric(pred_test,self.test_data[self.binary_target],mode=mod)}
                    
                    #Test records of given model added to trial logs
                    trial_log= pd.concat([trial_log,pd.DataFrame.from_dict(test_records)])
                    
                    #Selected features, train and test predictions of given model recorded to dictionary
                    model_dicts= {"feat":features,"train_pred":pred_train,"test_pred":pred_test}
                    
                    #This model dicts saved into another dictionary within given model and feature set name
                    model_preds["{}_{}".format(m,feat)]=model_dicts
                    
                    #If record option is set to be true, records dictionary saved with given model object and feature columns
                    if record:
                        records["Model_Obj"]=fitted_model
                        records["feats"]=features
                        with open("{}/{}_{}".format(record_dir,m,feat),"wb") as rc:
                            
                            pickle.dump(records,rc)
                            
                            
        # This method outputs, trial log of given model and their prediction on both train and test data.           
        return trial_log,model_preds
    
    
    def combinations(self,iterable, r):
        #Method for created desired number of model combinations
        
        pool = tuple(iterable)
        n = len(pool)
        
        if r > n:
            return
        indices = list(range(r))
        yield tuple(pool[i] for i in indices)
        while True:
            for i in reversed(range(r)):
                if indices[i] != i + n - r:
                    break
            else:
                return
            indices[i] += 1
            for j in range(i+1, r):
                indices[j] = indices[j-1] + 1
            yield tuple(pool[i] for i in indices)
          
    def ensemble(self,trial_log,model_preds,combs=[2]):
        
        #trial_log: trial log of single models 
        #Model_preds: prediction of single models.
        #combs: Desired number of combinations. Multiple number of combinations can be add. Ex:[3,5,6]
        
        
        #Itarated over combs list
        for comb in combs:
            
            #Model names retrieved from the class
            model_names= list(self.models["binary"].keys())+list(self.models["poisson"].keys())
            
            #outcome combinations method created combination of models with desired comb. number
            for i in self.combinations(model_names,comb):
                
                records={}
                
                #Iterated over each feature columns combinations
                for feat in self.features_dict.keys():
                    
                    #Model name recorded as list of selected models
                    records["Model_Name"]=[[k for k in i]]
                    
                    #Type specified as ensemble of desired combination number. Ex: "Ensemble_2"
                    records["type"]=["Ensemble_{}".format(comb)]
                    
                    #Key name of feature columns set
                    records["feat_select"]=[feat]
                    
                    #Binary target variable selected for ensembles
                    records["Target_Dist"]=["binary"]
                    
                    #Predictive power of ensembles tested on test data
                    records["Data_Type"]=["Test"]
                    
                    
                    
                    test_pred=np.array(0)
                    for k in i:
                        #For each model in combination, their models test data predictions summed
                        test_pred=test_pred+model_preds["{}_{}".format(k,feat)]["test_pred"]
                    
                    #Final prediction of ensemble calculated via averaging sum of test prediction of models.
                    final_test=test_pred/len(i)
                    
                    #Negative and positive recall of ensemble prediction calculated.
                    records={**records,**self.metric(final_test,self.test_data["Over_2.5"],mode="binary")}
                    
                #Results of given ensemble model added to trial log dataframe.
                trial_log= pd.concat([trial_log,pd.DataFrame.from_dict(records)])
        return trial_log
            
            

In [23]:
#Selected features loaded
with open("features.pkl","rb") as feat:
    features= pickle.load(feat)
    
    

In [25]:
#For each feature combinations away and home features joined into one list
feat_dict={}
feat_dict["manuel_selected"]=list(set(features["manuel_selected"]["home_features"]+features["manuel_selected"]["away_features"]))
feat_dict["test_selected"]=list(set(features["test_selected"]["home_features"]+features["test_selected"]["away_features"]))


In [26]:
#Uncessary columns for training and inference dropped.
train_v1=train.drop(["fixture.id","goals.home","goals.away","Full_Score"],axis=1)
test_v1=test.drop(["fixture.id","goals.home","goals.away","Full_Score"],axis=1)


In [27]:
#Betting model class initiliazed
bet_model= betting_model(train_v1,test_v1,
                         binary_target="Over_2.5",
                         features_dict=feat_dict,count_target= "Full_Score_adj")

In [28]:
#With initiliazed variables, each model in the bet_model object perform learning from train_v1 data
#Their positive and negative recall performance on test and training data saved into dataframe(trial_log)
#And their training and test data prediction saved into dictionary(model_preds)
#Each model saved with their learned parametter. (record_true)
trial_log,model_preds=bet_model.trials(record=True)

Log_Reg
all_columns


C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:63: RuntimeWarning: invalid value encountered in scalar divide
  pozitif_acc=pozitif_treshold.Target.sum()/pozitif_treshold.shape[0]
C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:65: RuntimeWarning: invalid value encountered in scalar divide
  negatif_acc=1-negatif_treshold.Target.sum()/negatif_treshold.shape[0]


manuel_selected


C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:65: RuntimeWarning: invalid value encountered in scalar divide
  negatif_acc=1-negatif_treshold.Target.sum()/negatif_treshold.shape[0]
C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:63: RuntimeWarning: invalid value encountered in scalar divide
  pozitif_acc=pozitif_treshold.Target.sum()/pozitif_treshold.shape[0]


test_selected


...

C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:65: RuntimeWarning: invalid value encountered in scalar divide
  negatif_acc=1-negatif_treshold.Target.sum()/negatif_treshold.shape[0]


manuel_selected


C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:65: RuntimeWarning: invalid value encountered in scalar divide
  negatif_acc=1-negatif_treshold.Target.sum()/negatif_treshold.shape[0]
C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:65: RuntimeWarning: invalid value encountered in scalar divide
  negatif_acc=1-negatif_treshold.Target.sum()/negatif_treshold.shape[0]


test_selected


C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:65: RuntimeWarning: invalid value encountered in scalar divide
  negatif_acc=1-negatif_treshold.Target.sum()/negatif_treshold.shape[0]
C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:65: RuntimeWarning: invalid value encountered in scalar divide
  negatif_acc=1-negatif_treshold.Target.sum()/negatif_treshold.shape[0]


In [29]:
#From single models, ensemble models tested with given number of combination.
#Their results saved within trial_log
trial_log_v1=bet_model.ensemble(trial_log,model_preds,[2,3])

C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:63: RuntimeWarning: invalid value encountered in scalar divide
  pozitif_acc=pozitif_treshold.Target.sum()/pozitif_treshold.shape[0]
C:\Users\Arda\AppData\Local\Temp\ipykernel_18612\1715804784.py:65: RuntimeWarning: invalid value encountered in scalar divide
  negatif_acc=1-negatif_treshold.Target.sum()/negatif_treshold.shape[0]


In [30]:
#Final trial log saved.
trial_log_v1.to_csv("trial_logs.csv",index=False)